<a href="https://colab.research.google.com/github/hardik1408/CS771/blob/hardik/d3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: open train_text_seq.csv

import pandas as pd
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')
df_train.head()



,input_str,label
0,0000154364642718159661428002624223132284159626...,0
1,0004641596369515436422262614110471596262476161...,0
2,0001543626215965999614422464135806142624051159...,0
3,0000154364224641238614262159689561596284351061...,1
4,0004641899422154362069015966142624761262159661...,1


In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Load the data
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Separate inputs and labels
X_train = df_train['input_str'].astype(str).values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].astype(str).values
y_valid = df_valid['label'].values

# Tokenize the sequences
tokenizer = Tokenizer(char_level=True)  # Character-level tokenization
tokenizer.fit_on_texts(X_train)

# Convert the sequences to integer format
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_valid_seq = tokenizer.texts_to_sequences(X_valid)

# Pad the sequences to ensure uniform length
max_len = 50  # Since the sequences are of length 50
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_valid_pad = pad_sequences(X_valid_seq, maxlen=max_len, padding='post')

# Define the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_valid_pad, y_valid))

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_pad, y_valid)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.5236 - loss: 0.6913 - val_accuracy: 0.5665 - val_loss: 0.6770
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.5864 - loss: 0.6715 - val_accuracy: 0.6196 - val_loss: 0.6516
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.6101 - loss: 0.6593 - val_accuracy: 0.6360 - val_loss: 0.6408
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.6293 - loss: 0.6445 - val_accuracy: 0.6299 - val_loss: 0.6437
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.6199 - loss: 0.6421 - val_accuracy: 0.6380 - val_loss: 0.6324
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.6285 - loss: 0.6335 - val_accuracy: 0.6421 - val_loss: 0.6296
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.6386 - loss: 0.6269 - val_accuracy: 0.6585 - val_loss: 0.6200
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.6420 - loss: 0.6294 - val_accuracy: 0.6

In [14]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 50, 32)              │             352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,749 (295.90 KB)

 Trainable params: 25,249 (98.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 50,500 (197.27 KB)

In [8]:
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_valid_pad, y_valid))


Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9404 - loss: 0.1614 - val_accuracy: 0.8119 - val_loss: 0.5577
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.9437 - loss: 0.1486 - val_accuracy: 0.8200 - val_loss: 0.5841
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9500 - loss: 0.1320 - val_accuracy: 0.7975 - val_loss: 0.6213
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9539 - loss: 0.1338 - val_accuracy: 0.7975 - val_loss: 0.6315
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9543 - loss: 0.1188 - val_accuracy: 0.7894 - val_loss: 0.6570
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9549 - loss: 0.1241 - val_accuracy: 0.7996 - val_loss: 0.6309
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9614 - loss: 0.1076 - val_accuracy: 0.8139 - val_loss: 0.6181
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9586 - loss: 0.1245 - val

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(X_valid_pad, y_valid)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8301 - loss: 0.6451
Validation Loss: 0.6425, Validation Accuracy: 0.8241


In [12]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 64)                  │          17,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,565 (228.77 KB)

 Trainable params: 19,521 (76.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 39,044 (152.52 KB)

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence))

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (50 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=50, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=50, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=32, input_length=50))  # 10 unique digits, embedding size 32
model.add(LSTM(64, return_sequences=False))  # LSTM layer with 64 units
model.add(Dense(32, activation='relu'))  # Dense layer with 32 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.5361 - loss: 0.6893 - val_accuracy: 0.6155 - val_loss: 0.6659
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.6064 - loss: 0.6556 - val_accuracy: 0.6380 - val_loss: 0.6404
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.6126 - loss: 0.6537 - val_accuracy: 0.6421 - val_loss: 0.6382
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.6293 - loss: 0.6411 - val_accuracy: 0.6483 - val_loss: 0.6374
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.6323 - loss: 0.6403 - val_accuracy: 0.6360 - val_loss: 0.6350
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 16s 60ms/step - accuracy: 0.6276 - loss: 0.6368 - val_accuracy: 0.6789 - val_loss: 0.6163
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.6397 - loss: 0.6306 - val_accuracy: 0.6319 - val_loss: 0.6166
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.6516 - loss: 0.6153 - val_accuracy: 

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 32)              │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 81,896 (319.91 KB)

 Trainable params: 27,298 (106.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 54,598 (213.28 KB)

In [3]:
# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=30, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 20s 88ms/step - accuracy: 0.6759 - loss: 0.5939 - val_accuracy: 0.6564 - val_loss: 0.6144
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.6771 - loss: 0.5867 - val_accuracy: 0.6789 - val_loss: 0.5880
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.6950 - loss: 0.5800 - val_accuracy: 0.6851 - val_loss: 0.5771
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.7019 - loss: 0.5630 - val_accuracy: 0.6994 - val_loss: 0.5638
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.7043 - loss: 0.5578 - val_accuracy: 0.6871 - val_loss: 0.5787
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.7292 - loss: 0.5320 - val_accuracy: 0.6892 - val_loss: 0.5738
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.7294 - loss: 0.5313 - val_accuracy: 0.6973 - val_loss: 0.5602
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.7351 - loss: 0.5146 - val_a

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence))

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (50 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=50, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=50, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model with reduced parameters
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=8, input_length=50))  # 10 unique digits, embedding size 8
model.add(LSTM(16, return_sequences=False))  # LSTM layer with 16 units
model.add(Dense(8, activation='relu'))  # Dense layer with 8 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.5150 - loss: 0.6928 - val_accuracy: 0.5481 - val_loss: 0.6866
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.5735 - loss: 0.6760 - val_accuracy: 0.5808 - val_loss: 0.6745
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.5990 - loss: 0.6575 - val_accuracy: 0.6094 - val_loss: 0.6433
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.6079 - loss: 0.6508 - val_accuracy: 0.6319 - val_loss: 0.6439
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.6152 - loss: 0.6459 - val_accuracy: 0.6319 - val_loss: 0.6426
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6263 - loss: 0.6437 - val_accuracy: 0.6278 - val_loss: 0.6376
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.6323 - loss: 0.6375 - val_accuracy: 0.6339 - val_loss: 0.6347
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.6315 - loss: 0.6373 - val_accuracy: 0.6

In [7]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 50, 8)               │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           1,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,504 (21.50 KB)

 Trainable params: 1,834 (7.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,670 (14.34 KB)

In [9]:
# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=50, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7203 - loss: 0.5435 - val_accuracy: 0.7219 - val_loss: 0.5782
Epoch 2/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.7227 - loss: 0.5304 - val_accuracy: 0.7157 - val_loss: 0.5760
Epoch 3/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.7311 - loss: 0.5307 - val_accuracy: 0.7260 - val_loss: 0.5710
Epoch 4/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7370 - loss: 0.5308 - val_accuracy: 0.7219 - val_loss: 0.5773
Epoch 5/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.7267 - loss: 0.5371 - val_accuracy: 0.7219 - val_loss: 0.5773
Epoch 6/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.7487 - loss: 0.5132 - val_accuracy: 0.7342 - val_loss: 0.5645
Epoch 7/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.7415 - loss: 0.5230 - val_accuracy: 0.7301 - val_loss: 0.5718
Epoch 8/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7329 - loss: 0.5337 - val_acc

In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence))

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (50 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=50, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=50, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model with approximately 10,000 trainable parameters
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=12, input_length=50))  # 10 unique digits, embedding size 12
model.add(LSTM(24, return_sequences=False))  # Bidirectional LSTM with 24 units
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(16, activation='relu'))  # Dense layer with 16 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - accuracy: 0.4979 - loss: 0.6933 - val_accuracy: 0.5194 - val_loss: 0.6918
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.5619 - loss: 0.6846 - val_accuracy: 0.6278 - val_loss: 0.6477
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.6147 - loss: 0.6516 - val_accuracy: 0.6237 - val_loss: 0.6420
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.6267 - loss: 0.6416 - val_accuracy: 0.6258 - val_loss: 0.6490
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.6206 - loss: 0.6423 - val_accuracy: 0.6380 - val_loss: 0.6374
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.6293 - loss: 0.6370 - val_accuracy: 0.6339 - val_loss: 0.6527
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.6318 - loss: 0.6406 - val_accuracy: 0.6503 - val_loss: 0.6260
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.6450 - loss: 0.6316 - val_accuracy: 0.

In [20]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, 50, 12)              │             120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 24)                  │           3,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,320 (48.13 KB)

 Trainable params: 4,106 (16.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,214 (32.09 KB)

In [25]:

model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


Epoch 1/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.8926 - loss: 0.2560 - val_accuracy: 0.7853 - val_loss: 0.4818
Epoch 2/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.8906 - loss: 0.2597 - val_accuracy: 0.8200 - val_loss: 0.4566
Epoch 3/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8896 - loss: 0.2587 - val_accuracy: 0.8057 - val_loss: 0.4756
Epoch 4/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8906 - loss: 0.2537 - val_accuracy: 0.7975 - val_loss: 0.4717
Epoch 5/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.8961 - loss: 0.2497 - val_accuracy: 0.7914 - val_loss: 0.5109
Epoch 6/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8954 - loss: 0.2447 - val_accuracy: 0.7955 - val_loss: 0.5150
Epoch 7/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.9017 - loss: 0.2395 - val_accuracy: 0.8037 - val_loss: 0.4973
Epoch 8/20
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8870 - loss: 0.2648 - val_accu

In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits, removing the first three zeroes
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence[3:]))  # Remove first 3 zeros

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (47 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=47, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=47, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model with more parameters while staying under 10,000
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=16, input_length=47))  # 10 unique digits, embedding size 16
model.add(LSTM(32, return_sequences=True))  # Bidirectional LSTM with 32 units, return sequences
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting
model.add(LSTM(16))  # Additional LSTM layer with 16 units
model.add(Dense(16, activation='relu'))  # Dense layer with 16 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=2, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')
model.summary()

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - accuracy: 0.5188 - loss: 0.6919 - val_accuracy: 0.5869 - val_loss: 0.6645
Epoch 2/2
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.6092 - loss: 0.6583 - val_accuracy: 0.6217 - val_loss: 0.6338
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6300 - loss: 0.6294
Validation Loss: 0.6338253021240234, Validation Accuracy: 0.6216768622398376


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 47, 16)              │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 47, 32)              │           6,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 47, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,624 (115.72 KB)

 Trainable params: 9,874 (38.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,750 (77.15 KB)

In [36]:
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=30, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

Epoch 1/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.9631 - loss: 0.0969 - val_accuracy: 0.8630 - val_loss: 0.6615
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9820 - loss: 0.0516 - val_accuracy: 0.8609 - val_loss: 0.6857
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9816 - loss: 0.0559 - val_accuracy: 0.8630 - val_loss: 0.6333
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.9762 - loss: 0.0630 - val_accuracy: 0.8569 - val_loss: 0.6887
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.9694 - loss: 0.0724 - val_accuracy: 0.8487 - val_loss: 0.6853
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9767 - loss: 0.0606 - val_accuracy: 0.8528 - val_loss: 0.6580
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9835 - loss: 0.0436 - val_accuracy: 0.8609 - val_loss: 0.7370
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.9728 - loss: 0.0670 - v